<a href="https://colab.research.google.com/github/dungdt-infopstats/TV-command-synthesis/blob/main/notebooks/DDSS_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

tridungdo_whisper_text_raw_df_json_path = kagglehub.dataset_download('tridungdo/whisper-text-raw-df-json')
tridungdo_wer_cut_res_path = kagglehub.dataset_download('tridungdo/wer-cut-res')
tridungdo_wer_025_raw_path = kagglehub.dataset_download('tridungdo/wer-025-raw')
tridungdo_transcribe_05_path = kagglehub.dataset_download('tridungdo/transcribe-05')

print('Data source import complete.')


# old wer, clean needed

In [ ]:
import pandas as pd

path = "/kaggle/input/wer-cut-res/asr_evaluation_results.csv"

df_07 = pd.read_csv(path)

In [ ]:
import pandas as pd
path_05 = "/kaggle/input/transcribe-05/asr_results.csv"
df_05 = pd.read_csv(path_05)
df_05 = df_05.rename(
    columns = {'text_active': 'reference_text'}
)

In [ ]:
df_new = df_07[~df_07['id_x'].isin(df_05['id_x'])]

df_05 = pd.concat([df_05, df_new], ignore_index = True)

In [ ]:
import pandas as pd

path_025 = "/kaggle/input/wer-025-raw/asr_evaluation_results_025.csv"

df_025 = pd.read_csv(path_025)

In [ ]:
import pandas as pd

import pandas as pd
from tqdm import tqdm

def get_transcribe(df, ref_col, reg_col, seg_col, id_col):
    results = []

    # group theo id_col, dùng tqdm để theo dõi tiến độ
    for id_val, group in tqdm(df.groupby(id_col), desc="Processing IDs"):
        # sắp xếp theo seg_col để đảm bảo đúng thứ tự
        group_sorted = group.sort_values(by=seg_col)

        ref_value = group_sorted[ref_col].iloc[0]  # lấy 1 cái là đủ
        reg_joined = " ".join(group_sorted[reg_col].astype(str).tolist())

        results.append({
            id_col: id_val,
            ref_col: ref_value,
            reg_col: reg_joined
        })

    return pd.DataFrame(results)


transcribed_df = get_transcribe(df = df_05,
                               ref_col = 'reference_text',
                               reg_col = 'recognized_text',
                               seg_col = 'segment_index',
                               id_col = 'id_x')

Processing IDs: 100%|██████████| 15834/15834 [00:09<00:00, 1599.73it/s]


In [ ]:
df_info = transcribed_df[['id_x', 'reference_text']] # true info with true length

# raw transcribe only

In [ ]:
import json
import pandas as pd

raw_path = "/kaggle/input/whisper-text-raw-df-json/whisper_text_raw_df.json"

with open(raw_path) as f:
    raw_data = json.load(f)

df_raw = pd.DataFrame(raw_data)
df_raw_final = pd.merge(df_raw, df_info, left_on = 'id', right_on = 'id_x')
df_raw_final = df_raw_final.rename(columns = {
    "full_text": "recognized_text"
})

# Preprocessing

In [ ]:
import pandas as pd
import re


def clean_text(text: str) -> str:
        # bỏ dấu câu
        text = re.sub(r"[^\w\s]", "", text)
        # thay nhiều khoảng trắng thành 1 khoảng trắng
        text = re.sub(r"\s+", " ", text)
        return text.strip()

def preprocessing(df: pd.DataFrame, ref_col: str, reg_col: str) -> pd.DataFrame:
    # Fillna bằng chuỗi rỗng
    df[ref_col] = df[ref_col].fillna("")
    df[reg_col] = df[reg_col].fillna("")


    df[ref_col] = df[ref_col].apply(clean_text)
    df[reg_col] = df[reg_col].apply(clean_text)

    return df

In [ ]:
################## COMPUTE_DF

COMPUTE_DF = transcribed_df

In [ ]:
preprocessed_df = preprocessing(
    df = COMPUTE_DF,
    ref_col = "reference_text",
    reg_col = "recognized_text"
)

# WER

In [ ]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 31.2 MB/s eta 0:00:0000:0100:01


In [ ]:
import pandas as pd
import jiwer
from tqdm import tqdm

def compute_wer(df: pd.DataFrame, ref_col: str, hyp_col: str) -> pd.DataFrame:
    """
    Tính toán WER metrics cho từng hàng trong DataFrame.
    - ref_col: cột chứa ground-truth text
    - hyp_col: cột chứa predicted text
    """

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Computing WER"):
        ref = str(row[ref_col]) if pd.notna(row[ref_col]) else ""
        hyp = str(row[hyp_col]) if pd.notna(row[hyp_col]) else ""

        output = jiwer.process_words(reference=ref, hypothesis=hyp).__dict__

        # Đảm bảo các cột metrics đã tồn tại
        for key, val in output.items():
            if key not in df.columns:
                df[key] = None
            df.at[idx, key] = val

        # Đếm số từ trong reference
        words_count = len(ref.split()) if ref.strip() else 0
        if "words_count" not in df.columns:
            df["words_count"] = None
        df.at[idx, "words_count"] = words_count

    return df


In [ ]:
df_wer = compute_wer(
    df = preprocessed_df,
    ref_col = "reference_text",
    hyp_col = "recognized_text"
)

Computing WER: 100%|██████████| 15834/15834 [00:04<00:00, 3354.26it/s]


In [ ]:
import numpy as np

def statistics(df_wer):
    res = {}
    res["N"] = np.nansum(df_wer['words_count'])

    res["D"] = np.nansum(df_wer['deletions'])
    res["I"] = np.nansum(df_wer['insertions'])
    res["S"] = np.nansum(df_wer['substitutions'])

    res["D_rate"] = res["D"] / res["N"]
    res["I_rate"] = res["I"] / res["N"]
    res["S_rate"] = res["S"] / res["N"]
    res["WER"] = (res["D"] + res["I"] + res["S"]) / res["N"]

    return res

In [ ]:
df_05 = statistics(df_wer)

In [ ]:
def pipeline(df, ref_col, reg_col, seg_col, id_col):
    try:
        output = get_transcribe(df = df,
                                   ref_col = ref_col,
                                   reg_col = reg_col,
                                   seg_col = seg_col,
                                   id_col = id_col)
    except:
        print('NO TRANSCRIBE')
        output = df

    output = preprocessing(
        df = output,
        ref_col = ref_col,
        reg_col = reg_col
    )

    df_res = compute_wer(
        df = output,
        ref_col = ref_col,
        hyp_col = reg_col
    )
    stats = statistics(df_res)
    return df_res, stats

In [ ]:
list_df = {
    '025': df_025,
    '05': df_05,
    '07': df_07,
    'raw': df_raw_final
}
ref_col = 'reference_text'
reg_col = 'recognized_text'
seg_col = 'segment_index'
id_col = 'id_x'

list_out = {}
for name in list_df:
    res, stats = pipeline(
        df = list_df[name],
        ref_col = ref_col,
        reg_col = reg_col,
        seg_col = seg_col,
        id_col = id_col,
    )

    list_out[name] = {
        'res': res,
        'stats': stats
    }

Processing IDs:   0%|          | 0/15834 [00:00<?, ?it/s]


NO TRANSCRIBE


Computing WER: 100%|██████████| 15834/15834 [00:04<00:00, 3407.97it/s]


In [ ]:
for i in list_out:
    print(i)
    print(list_out[i]['stats'])

025
{'N': 77350, 'D': 8277, 'I': 42660, 'S': 22501, 'D_rate': 0.1070071105365223, 'I_rate': 0.551519069166128, 'S_rate': 0.29089851325145444, 'WER': 0.9494246929541047}
05
{'N': 77350, 'D': 9993, 'I': 30497, 'S': 21328, 'D_rate': 0.12919198448610214, 'I_rate': 0.39427278603749194, 'S_rate': 0.27573367808661925, 'WER': 0.7991984486102133}
07
{'N': 77350, 'D': 12708, 'I': 18991, 'S': 20604, 'D_rate': 0.16429217840982546, 'I_rate': 0.24552036199095023, 'S_rate': 0.2663736263736264, 'WER': 0.676186166774402}
raw
{'N': 77350, 'D': 4013, 'I': 181419, 'S': 23786, 'D_rate': 0.05188106011635423, 'I_rate': 2.3454298642533935, 'S_rate': 0.3075113122171946, 'WER': 2.7048222365869425}
